<a href="https://colab.research.google.com/github/zhaoheng001/colabtogithubdemo/blob/master/GBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import math
import tensorflow as tf
# Importing some more libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
h = 0.01
#save last pooling layer with a dim 512 vector 
all_images_data = pd.read_csv('active_all_flt.csv')
all_images_data = all_images_data.loc[:, 'label':'pixel512']
sample_class0 = all_images_data[all_images_data.label == 0]
sample_class1 = all_images_data[all_images_data.label == 1]
sample_class0 = sample_class0.loc[1:960]
sample_class1 = sample_class1.loc[1:1100]
frames = [sample_class0, sample_class1]
sample200 = pd.concat(frames)

from sklearn.utils import shuffle
sample200 = shuffle(sample200)
sample200.index = range(len(sample200))
sample_x = sample200.loc[:, 'pixel1':'pixel512']
sample_y = sample200.loc[:, 'label']
sample_x = sample_x.values
#sample_y = tf.to_int32(sample_y, name='ToInt32')
#to one hot
sample_y_list = list(sample_y)
sample_y_int = map(int, sample_y_list)
sample_y_binary = tf.one_hot(sample_y_int, depth=2)
with tf.Session() as sess:
    sample_y_bi = sess.run(sample_y_binary)

In [0]:
def accuracy(Input_x,Input_y,weights):
    accnum = 0
    for i in range(20):
        In_node = Input_x[i]
        Hid_inil = np.random.randint(0,2,(202))
        Threshold = [1]
        allnode = np.hstack((In_node,Hid_inil))
        allnode = np.hstack((allnode,Threshold))
        for j in range(202):
            Hidn_nd = np.dot(weights[j+512], allnode)
            Hidn_nd = 1.0/(1.0+math.exp(-Hidn_nd))
            p = np.array([1.0 - Hidn_nd, Hidn_nd])
            Hid_inil[j] = np.random.choice([0, 1], p = p.ravel())
            Hidn_node1 = Hid_inil
        
        
        allnode = np.hstack((In_node,Hidn_node1))  
        allnode = np.hstack((allnode,Threshold))
        for j in range(202):
            Hidn_nd = np.dot(weights[j+512], allnode)
            Hidn_nd = 1.0/(1.0+math.exp(-Hidn_nd))
            p = np.array([1.0 - Hidn_nd, Hidn_nd])
            Hid_inil[j] = np.random.choice([0, 1], p = p.ravel())
            Hidn_node2 = Hid_inil
            
        allnode = np.hstack((In_node,Hidn_node2))  
        allnode = np.hstack((allnode,Threshold))
        for j in range(202):
            Hidn_nd = np.dot(weights[j+512], allnode)
            Hidn_nd = 1.0/(1.0+math.exp(-Hidn_nd))
            p = np.array([1.0 - Hidn_nd, Hidn_nd])
            Hid_inil[j] = np.random.choice([0, 1], p = p.ravel())
            Hidn_node3 = Hid_inil
            
            
        allnode = np.hstack((In_node,Hidn_node3))  
        allnode = np.hstack((allnode,Threshold))
        for j in range(202):
            Hidn_nd = np.dot(weights[j+512], allnode)
            Hidn_nd = 1.0/(1.0+math.exp(-Hidn_nd))
            p = np.array([1.0 - Hidn_nd, Hidn_nd])
            Hid_inil[j] = np.random.choice([0, 1], p = p.ravel())
            Hidn_node4 = Hid_inil
            
            
        allnode = np.hstack((In_node,Hidn_node4))  
        allnode = np.hstack((allnode,Threshold))
        for j in range(202):
            Hidn_nd = np.dot(weights[j+512], allnode)
            Hidn_nd = 1.0/(1.0+math.exp(-Hidn_nd))
            p = np.array([1.0 - Hidn_nd, Hidn_nd])
            Hid_inil[j] = np.random.choice([0, 1], p = p.ravel())
            Hidn_node5 = Hid_inil
        
        pred_outnod1 = np.dot(weights[712], allnode)
        pred_outnod1 = 1.0/(1.0+math.exp(-pred_outnod1))
        
        pred_outnod2 = np.dot(weights[713], allnode)
        pred_outnod2 = 1.0/(1.0+math.exp(-pred_outnod2))
        
        prob1 = pred_outnod1/(pred_outnod1 + pred_outnod2)
        prob2 = pred_outnod2/(pred_outnod1 + pred_outnod2)
        pred_y = [prob1, prob2]
        
        
        #pred_y = Hidn_node5[200:202]
        pred_y = np.argmax(pred_y)
        real_y = np.argmax(Input_y[i])
        if pred_y == real_y:
            accnum = accnum + 1
    acc = accnum/(20.0)
    return acc

In [0]:
def train(In, Out, epoch):
    h = 0.01
    weight_ini = np.random.random(size=(715,715))-0.5
    w = (weight_ini + weight_ini.T)/2
    for i in range(512):
        for j in range(512):
            w[i][j] = 0
    for i in range(512,712):
        for j in range(512,712):
            w[i][j] = 0
    for i in range(712,714):
        for j in range(712,714):
            w[i][j] = 0
    for i in range(512):
        for j in range(712,715):
            w[i][j] = 0
    for i in range(712,715):
        for j in range(512):
            w[i][j] = 0
        #w[j][i] = 0
    w[714][714] = 0
    #M_inil = np.zeros((200,715,715))
    M = np.zeros((715,715))
    
    for i in range(epoch):
        
        for j in range(20):
            #Hid_inil = np.random.randint(0,2,(200))
            In_node = In[j]
            #Hid_node = Hid[j]
            Hid_inil = np.random.randint(0,2,(200))
            Out_node = Out[j]
            Threshold = [1]
            allnode = np.hstack((In_node,Hid_inil))  
            allnode = np.hstack((allnode,Out_node))
            allnode = np.hstack((allnode,Threshold))
            
            #initial Hidden layer for each input
            
            for k in range(200):
                Hidn_nd = np.dot(w[k+512], allnode)
                Hidn_nd = 1.0/(1.0+math.exp(-Hidn_nd))
                p = np.array([1.0 - Hidn_nd, Hidn_nd])
                
                Hid_inil[k] = np.random.choice([0, 1], p = p.ravel())
                Hidn_node1 = Hid_inil
            
            allnode = np.hstack((In_node,Hidn_node1))  
            allnode = np.hstack((allnode,Out_node))
            allnode = np.hstack((allnode,Threshold))
                
            for k in range(200):     
                Hidn_nd = np.dot(w[k+512], allnode)
                Hidn_nd = 1.0/(1.0+math.exp(-Hidn_nd))
                p = np.array([1.0 - Hidn_nd, Hidn_nd])
                
                Hid_inil[k] = np.random.choice([0, 1], p = p.ravel())
                Hidn_node2 = Hid_inil
                
          
            allnode = np.hstack((In_node,Hidn_node2))  
            allnode = np.hstack((allnode,Out_node))
            allnode = np.hstack((allnode,Threshold))
        
            for k in range(200):     
                Hidn_nd = np.dot(w[k+512], allnode)
                Hidn_nd = 1.0/(1.0+math.exp(-Hidn_nd))
                p = np.array([1.0 - Hidn_nd, Hidn_nd])
                
                Hid_inil[k] = np.random.choice([0, 1], p = p.ravel())
                Hidn_node3 = Hid_inil
                
                
            clp = np.zeros((715,715))
            for l in range(512,712):
                for m in range(512):
                    clp[l][m] = np.dot(Hidn_node1[l-512], In_node[m]) + np.dot(Hidn_node2[l-512], In_node[m]) + np.dot(Hidn_node3[l-512], In_node[m])
                for m in range(712,714):
                    clp[l][m] = np.dot(Hidn_node1[l-512], Out_node[m-712]) + np.dot(Hidn_node2[l-512], Out_node[m-712]) +np.dot(Hidn_node3[l-512], Out_node[m-712])
                clp[l][714] = Hidn_node1[l-512] + Hidn_node2[l-512] + Hidn_node3[l-512]
            clp = clp + clp.T
              
            
            
            
            #initial Hidden layer for each input
            Hid_inil = np.random.randint(0,2,(202))
            allnode = np.hstack((In_node,Hid_inil))  
            allnode = np.hstack((allnode,Threshold))
            for k in range(202):
                Hidn_nd = np.dot(w[k+512], allnode)
                Hidn_nd = 1.0/(1.0+math.exp(-Hidn_nd))
                p = np.array([1.0 - Hidn_nd, Hidn_nd])
                
                Hid_inil[k] = np.random.choice([0, 1], p = p.ravel())
                Hidn_node1 = Hid_inil
                
            allnode = np.hstack((In_node,Hidn_node1))  
            allnode = np.hstack((allnode,Threshold))
                
            for k in range(200):     
                Hidn_nd = np.dot(w[k+512], allnode)
                Hidn_nd = 1.0/(1.0+math.exp(-Hidn_nd))
                p = np.array([1.0 - Hidn_nd, Hidn_nd])
                
                Hid_inil[k] = np.random.choice([0, 1], p = p.ravel())
                Hidn_node2 = Hid_inil
                
          
            allnode = np.hstack((In_node,Hidn_node2))  
            allnode = np.hstack((allnode,Threshold))
        
            for k in range(200):     
                Hidn_nd = np.dot(w[k+512], allnode)
                Hidn_nd = 1.0/(1.0+math.exp(-Hidn_nd))
                p = np.array([1.0 - Hidn_nd, Hidn_nd])
                
                Hid_inil[k] = np.random.choice([0, 1], p = p.ravel())
                Hidn_node3 = Hid_inil
        
        
            unclp = np.zeros((715,715))
            for l in range(512,712):
                for m in range(512):
                    unclp[l][m] = np.dot(Hidn_node1[l-512], In_node[m]) + np.dot(Hidn_node2[l-512], In_node[m]) + np.dot(Hidn_node3[l-512], In_node[m])
                for m in range(712,714):
                    unclp[l][m] = np.dot(Hidn_node1[l-512], Out_node[m-712]) + np.dot(Hidn_node2[l-512], Out_node[m-712]) +np.dot(Hidn_node3[l-512], Out_node[m-712])
                unclp[l][714] = Hidn_node1[l-512] + Hidn_node2[l-512] + Hidn_node3[l-512]
            unclp = unclp + unclp.T
            
            
            M0 = clp - unclp
            M = M + M0
        M = M/(20.0)
        print(np.linalg.norm(M,ord=2))
        w = w + h*M
        Accuracy = accuracy(In, Out, w)
        print(Accuracy)
        

In [26]:
train(sample_x, sample_y_bi, 20)

122.11866741247346
0.6
92.08521252503921
0.55
90.35636439082018
0.55
72.49548519583237
0.55
81.0672519557679
0.55
74.3452020985663
0.45
60.88977019122126
0.25
62.39212988819856
0.15
78.10121709344716
0.3
52.4642785187536
0.35
54.7713159764017
0.35
54.41552878824171
0.55
69.79057890428004
0.35
59.05783707412071
0.25
45.101556086562645
0.3
55.001432731188196
0.5
65.26874874024003
0.7
57.139890735722176
0.7
47.79284576010238
0.55
44.972776029382594
0.6
